In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
base_path = r'D:\Object Detection\Blood\Data'
annotation_path = os.path.join(base_path,'Annotations')
train_folders_path = os.path.join(base_path,'train')
validation_folders_path = ''
test_folders_path = os.path.join(base_path,'test')

## Pascal VOC XML  >  CSV

In [3]:
import glob
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):               # search for any files ending with .xml in 'path'
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [4]:
xml_df = xml_to_csv(annotation_path)
xml_df

,filename,width,height,class,xmin,ymin,xmax,ymax
0,BloodImage_00000,640,480,RBC,216,359,316,464
1,BloodImage_00000,640,480,RBC,77,326,177,431
2,BloodImage_00000,640,480,RBC,540,353,640,458
3,BloodImage_00000,640,480,RBC,405,350,513,457
4,BloodImage_00000,640,480,RBC,160,72,245,177
...,...,...,...,...,...,...,...,...
3849,BloodImage_00410,640,480,RBC,1,221,108,322
3850,BloodImage_00410,640,480,RBC,582,379,639,480
3851,BloodImage_00410,640,480,RBC,457,372,562,456
3852,BloodImage_00410,640,480,RBC,563,20,640,113


In [5]:
xml_df.to_csv(os.path.join(base_path,'annotations.csv'), index=None)
print('Successfully converted xml to csv.')

Successfully converted xml to csv.


## Split data into train/val/test

In [6]:
full_annotations = pd.read_csv(os.path.join(base_path,'annotations.csv'))
full_annotations

,filename,width,height,class,xmin,ymin,xmax,ymax
0,BloodImage_00000,640,480,RBC,216,359,316,464
1,BloodImage_00000,640,480,RBC,77,326,177,431
2,BloodImage_00000,640,480,RBC,540,353,640,458
3,BloodImage_00000,640,480,RBC,405,350,513,457
4,BloodImage_00000,640,480,RBC,160,72,245,177
...,...,...,...,...,...,...,...,...
3849,BloodImage_00410,640,480,RBC,1,221,108,322
3850,BloodImage_00410,640,480,RBC,582,379,639,480
3851,BloodImage_00410,640,480,RBC,457,372,562,456
3852,BloodImage_00410,640,480,RBC,563,20,640,113


In [7]:
gb = full_annotations.groupby('filename')

In [8]:
grouped_list = [gb.get_group(x) for x in gb.groups]
train_index = np.random.choice(len(grouped_list), size=int(len(grouped_list)*0.7), replace=False)
test_index = np.setdiff1d(list(range(200)), train_index)
len(train_index), len(test_index)

(240, 57)

In [9]:
train = pd.concat([grouped_list[i] for i in train_index])
test = pd.concat([grouped_list[i] for i in test_index])
len(train), len(test)

(2679, 642)

In [10]:
train.to_csv(os.path.join(train_folders_path, 'train_annotations.csv'), index=None)
test.to_csv(os.path.join(test_folders_path, 'test_annotations.csv'), index=None)

## Generate TFRecords

In [12]:
import io
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict


# TO-DO replace this with label map
def class_text_to_int(row_label):                # Change class A,B,C,D,E,....... according to the classes we have
    if row_label == 'RBC':
        return 1
#    if row_label == 'class B':
#        return 2
#    if row_label == 'class C':
#        return 3
#    if row_label == 'class D':
#        return 4
#    if row_label == 'class E':
#        return 5


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}.jpg'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def train_TFRecords():
    writer = tf.io.TFRecordWriter(os.path.join(train_folders_path, 'train.record'))
    imagepath = os.path.join(base_path, 'Images')
    examples = pd.read_csv(os.path.join(train_folders_path, 'train_annotations.csv'))
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, imagepath)
        writer.write(tf_example.SerializeToString())

    writer.close()
    print('Successfully created the TFRecords at {}'.format(train_folders_path))
    
def test_TFRecords():
    writer = tf.io.TFRecordWriter(os.path.join(test_folders_path, 'test.record'))
    imagepath = os.path.join(base_path, 'Images')
    examples = pd.read_csv(os.path.join(test_folders_path, 'test_annotations.csv'))
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, imagepath)
        writer.write(tf_example.SerializeToString())

    writer.close()
    print('Successfully created the TFRecords at {}'.format(test_folders_path))
train_TFRecords()
test_TFRecords()

Successfully created the TFRecords at D:\Object Detection\Blood\Data\train
Successfully created the TFRecords at D:\Object Detection\Blood\Data\test
